## **Data Cleaning** File: Student_2020_Responses  
Objective: Skill Gaps Between International and Domestic Students

In [ ]:
# Import pandas library for data manipulation
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Required sheets from the Excel file
file_student = "/content/drive/MyDrive/M.S. BUSINESS ANALYTICS/MIS612/CO-OP SURVEY GROUP PROJECT/Student_2020_Responses.xlsx"
sheets_to_read = ["Classroom Activities", "College Specific Questions"]

In [ ]:
# Read only the necessary sheets from the Excel
# Setting header=1 to indicate that the actual column names are in the second row of the sheet
sheets_student = pd.read_excel(file_student, sheet_name=sheets_to_read, header=1)

In [ ]:
# Keep specific columns, on the "College Specific Questions" sheet,
college_specific_questions_df = sheets_student["College Specific Questions"][
    ["Responder ID", "Terms", "Coll", "Major", "Conc", "Class BOT", "Citizenship", "Coop #",
     "What skills do you need to develop or improve to better prepare you for your next position?"]
]

In [ ]:
# Fill missing text entries with "No Response" to ensure no rows are dropped due to missing values.
college_specific_questions_df.fillna("No Response", inplace=True)

<ipython-input-18-fc94bd2476d0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  college_specific_questions_df.fillna("No Response", inplace=True)


In [ ]:
# Apply the transformation only to the specific column
column_name = "What skills do you need to develop or improve to better prepare you for your next position?"
college_specific_questions_df[column_name] = college_specific_questions_df[column_name].str.lower().str.strip().str.replace(r'[^a-z0-9\s]', '', regex=True)

<ipython-input-19-23584ac06568>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  college_specific_questions_df[column_name] = college_specific_questions_df[column_name].str.lower().str.strip().str.replace(r'[^a-z0-9\s]', '', regex=True)


In [ ]:
# Keep specific columns on the "Classroom Activities" sheet,
classroom_activities_df = sheets_student["Classroom Activities"][
    ["Responder ID", "Terms", "Coll", "Major", "Conc", "Class BOT", "Citizenship", "Coop #",
     "Did your classroom activities prepare you for co-op?",
     "How did your classroom activities prepare you for co-op? If they didn’t, how were you prepared for co-op?"]
]

In [ ]:
# Fill missing text entries with "No Response" to ensure no rows are dropped due to missing values.
classroom_activities_df.fillna("No Response", inplace=True)

<ipython-input-21-a1dd047f7fcc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  classroom_activities_df.fillna("No Response", inplace=True)


In [ ]:
# Save the cleaned data to a new Excel file
output_file_path = "/content/drive/MyDrive/M.S. BUSINESS ANALYTICS/MIS612/CO-OP SURVEY GROUP PROJECT/Cleaned_Data/Cleaned_Student_2020_Responses.xlsx"
with pd.ExcelWriter(output_file_path) as writer:
    # Write the cleaned "College Specific Questions" sheet
    college_specific_questions_df.to_excel(writer, sheet_name="College Specific Questions", index=False)

# Output the path to the cleaned Excel file
output_file_path

'/content/drive/MyDrive/M.S. BUSINESS ANALYTICS/MIS612/CO-OP SURVEY GROUP PROJECT/Cleaned_Data/Cleaned_Student_2020_Responses.xlsx'

## **Data Cleaning** File: Employer_2020_Responses  
Objective: Skill Gaps Between International and Domestic Students

In [ ]:
# Import pandas library for data manipulation
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the required sheets from the Excel file
file_employer = "/content/drive/MyDrive/M.S. BUSINESS ANALYTICS/MIS612/CO-OP SURVEY GROUP PROJECT/Employer_2020_Responses.xlsx"
sheets_to_read = ["Weaknesses"]

In [ ]:
# Read only the necessary sheets from the Excel file using the first row as headers
# Setting header=1 to indicate that the actual column names are in the second row of the sheet
sheets_employer = pd.read_excel(file_employer, sheet_name=sheets_to_read, header=1)

In [ ]:
# Drop the "Strenghts", "Academic Prep" and "College Specific Questions" sheets by not loading them initially
# Only the sheets "Classroom Activities" and "College Specific Questions" are read for further processing

# On the "Weaknesses" sheet, keep specific columns
weaknesses_df = sheets_employer["Weaknesses"][
    ["Responder ID", "Terms", "Coll", "Major", "Conc 1", "Class BOT", "Citizenship", "Coop #",
     "What are the student's weaknesses/areas for improvement?"]
]

In [ ]:
# Fill missing text entries with "No Response" to ensure no rows are dropped due to missing values.
weaknesses_df.fillna("No Response", inplace=True)

In [ ]:
# Apply the transformations only to the specified column
weaknesses_df["What are the student's weaknesses/areas for improvement?"] = weaknesses_df["What are the student's weaknesses/areas for improvement?"].str.lower().str.strip().str.replace(r'[^a-z0-9\s]', '', regex=True)


In [ ]:
# Save the cleaned data to a new Excel file
# Creating a new Excel file to store the cleaned data with the selected columns
output_file_path = "/content/drive/MyDrive/M.S. BUSINESS ANALYTICS/MIS612/CO-OP SURVEY GROUP PROJECT/Cleaned_Data/Cleaned_Employer_2020_Responses.xlsx"
with pd.ExcelWriter(output_file_path) as writer:
    # Write the cleaned "Weaknesses" sheet
    weaknesses_df.to_excel(writer, sheet_name="Weaknesses", index=False)

# Output the path to the cleaned Excel file
output_file_path

'/content/drive/MyDrive/M.S. BUSINESS ANALYTICS/MIS612/CO-OP SURVEY GROUP PROJECT/Cleaned_Data/Cleaned_Employer_2020_Responses.xlsx'

## **Text Mining** File: Cleaned_Student_2020_Responses  
Objective: Skill Gaps Between International and Domestic Students

KPI 1: Skills for Improvement, by students




**Step 4 - Extract Common Skills by students - for KPI 1

In [ ]:
import pandas as pd
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords
from transformers import BertTokenizer, BertModel
import torch

# Download the stopwords from NLTK
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Load the cleaned data from the provided Excel file
file_student = "/content/drive/MyDrive/M.S. BUSINESS ANALYTICS/MIS612/CO-OP SURVEY GROUP PROJECT/Cleaned_Data/Cleaned_Student_2020_Responses.xlsx"
sheets_to_read = ["College Specific Questions"]

# Read only the necessary sheet from the Excel file using the first row as headers
sheets_student = pd.read_excel(file_student, sheet_name=sheets_to_read, header=0)
college_specific_questions_df = sheets_student["College Specific Questions"]

# Print the column names to verify
print(college_specific_questions_df.columns)

# Extract the correct column: "What skills do you need to develop or improve to better prepare you for your next position?"
column_name = "What skills do you need to develop or improve to better prepare you for your next position?"
improve_skills_responses = college_specific_questions_df[column_name].dropna().astype(str).tolist()

# Generate embeddings using BERT for contextual understanding
# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Tokenize and encode the responses using BERT tokenizer
encoded_inputs = tokenizer(
    improve_skills_responses,
    padding=True,
    truncation=True,
    return_tensors='pt'
)

# Generate embeddings using BERT
with torch.no_grad():
    outputs = model(**encoded_inputs)
    # Use the CLS token representation as the embedding for each response
    embeddings = outputs.last_hidden_state[:, 0, :]

# Convert embeddings from GPU tensor to CPU if needed
embeddings_cpu = embeddings.cpu().numpy() if embeddings.is_cuda else embeddings.numpy()

# Analyze responses to identify common skills, excluding stopwords and specific irrelevant terms
excluded_words = {'still','understanding','often', 'little','great','one','however','give','come','see','project','projects', 'take','takes','taking','sometime','sometimes','x000d','team','skills', 'skill','student','area','areas','weakness','weaknesses','improve','improvement', 'work', 'improve', 'need','needs','continue','experience','learn', 'enhance', 'develop', 'would', 'coop', 'response', 'error', 'better', 'think', 'position', 'next', 'should', 'like', 'want', 'will', 'feel', 'could', 'also', 'get', 'management','co','op','knowledge','able','time','well','help','working','always','thing','prepare','things','something','somethings','different'}
combined_phrases = ["critical thinking","project management", "time management", "attention to detail", "public speaking"]

# Define synonyms for common phrases to be treated as equivalent
data_analysis_skills = ["data analysis", "data analytics", "analyzing data", "data examination","analytical skills","data","analytical"]
finance_skills = ["financial analysis","financial", "finance", "budgeting", "financial planning", "accounting"]
problem_solving_skills = ["negotiation","logical reasoning","reasoninig","creative thinking","problem solving", "critical thinking", "troubleshooting", "attention to detail", "analytical thinking"]
ask_questions_skills = ["help","asking","ask","ask questions","ask more questions","ask question","asking questions","asking a","questions","question"]
communication_skills = ["talking","talk","communication","verbal communication","written communication","verbal","written","communication skills","express","writing skills","writing","communications"]
presentation_skills = ["speaking","speak","presenter","story telling","storytelling","public speaking","presenting","present","presentation"]
give_new_ideas = ["thoughts","convey my thoughts","convey","thoughts","new ideas","new idea","ideas","give ideas","idea","suggestion","new suggestions","suggestions"]
confidence = ["confidence","confident","confidently","confidence skills"]
attention_to_detail = ["listening","active listening", "actively listening","attention to detail","attention","details","detail","attention to","detail oriented","detailoriented"]
task_completion = ["finish","finish projects","complete task","complete tasks","task completion","task","tasks","following instruction","following instructions","instructions"]
organizational_skills = ["organizational skills","organized","organization","organize","project management"]
marketing = ["market research","creative","marketing courses","marketing","marketing skills","social media",]
technical_skills = ["research skills","research","technical skills","technical",]
programming_skills = ["technological","computer skills","computer", "tableau", "sap","scripting","microsoft","web design","coding","software","softwares","python","sql","power bi","programming"]
time_management = ["time efficiency","time management","better time","time"]
taking_initiative = ["taking initiative","taking initiatives","initiative","proactive","initiatives"]

# Update combined phrases list to include more groupings
combined_phrases.extend(data_analysis_skills)
combined_phrases.extend(finance_skills)
combined_phrases.extend(problem_solving_skills)
combined_phrases.extend(ask_questions_skills)
combined_phrases.extend(communication_skills)
combined_phrases.extend(presentation_skills)
combined_phrases.extend(give_new_ideas)
combined_phrases.extend(confidence)
combined_phrases.extend(attention_to_detail)
combined_phrases.extend(task_completion)
combined_phrases.extend(organizational_skills)
combined_phrases.extend(marketing)
combined_phrases.extend(technical_skills)
combined_phrases.extend(programming_skills)
combined_phrases.extend(time_management)
combined_phrases.extend(taking_initiative)

# Tokenize all responses to find common words and phrases
all_words = []
for response in improve_skills_responses:
    # Replace combined phrases first
    response = response.lower()
    for phrase in combined_phrases:
        response = re.sub(rf'\b{re.escape(phrase)}\b', phrase.replace(' ', '_'), response)  # Replace combined phrases with underscores
    # Replace synonyms for data analysis
    for synonym in data_analysis_skills:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'data_analysis_skills', response)
    for synonym in finance_skills:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'finance_skills', response)
    for synonym in problem_solving_skills:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'problem_solving_skills', response)
    for synonym in ask_questions_skills:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'ask_questions_skills', response)
    for synonym in communication_skills:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'communication_skills', response)
    for synonym in presentation_skills:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'presentation_skills', response)
    for synonym in give_new_ideas:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'give_new_ideas', response)
    for synonym in confidence:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'confidence', response)
    for synonym in attention_to_detail:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'attention_to_detail_skills', response)
    for synonym in task_completion:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'task_completion', response)
    for synonym in organizational_skills:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'organizational_skills', response)
    for synonym in marketing:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'marketing', response)
    for synonym in technical_skills:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'technical_skills', response)
    for synonym in programming_skills:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'programming_skills', response)
    for synonym in time_management:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'time_management', response)
    for synonym in taking_initiative:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'taking_initiative_skills', response)
    # Tokenize the response
    words = re.findall(r'\b\w+_?\w*\b', response)  # Extract all words and combined phrases
    filtered_words = [word for word in words if word not in stop_words and word not in excluded_words]
    # Remove subset terms if the grouped term is already present
    if 'finance_skills' in filtered_words:
        filtered_words = [word for word in filtered_words if word not in finance_skills]
    if 'data_analysis_skills' in filtered_words:
        filtered_words = [word for word in filtered_words if word not in data_analysis_skills]
    if 'problem_solving_skills' in filtered_words:
        filtered_words = [word for word in filtered_words if word not in problem_solving_skills]
    if 'ask_questions_skills' in filtered_words:
        filtered_words = [word for word in filtered_words if word not in ask_questions_skills]
    if  'communication_skills' in filtered_words:
        filtered_words = [word for word in filtered_words if word not in communication_skills]
    if 'presentation_skills' in filtered_words:
        filtered_words = [word for word in filtered_words if word not in presentation_skills]
    if 'give_new_ideas' in filtered_words:
        filtered_words = [word for word in filtered_words if word not in give_new_ideas]
    if 'confidence' in filtered_words:
        filtered_words = [word for word in filtered_words if word not in confidence]
    if  'attention_to_detail_skills' in filtered_words:
        filtered_words = [word for word in filtered_words if word not in attention_to_detail]
    if 'task_completion' in filtered_words:
        filtered_words = [word for word in filtered_words if word not in task_completion]
    if 'organizational_skills' in filtered_words:
        filtered_words = [word for word in filtered_words if word not in organizational_skills]
    if 'marketing' in filtered_words:
        filtered_words = [word for word in filtered_words if word not in marketing]
    if 'technical_skills' in filtered_words:
        filtered_words = [word for word in filtered_words if word not in technical_skills]
    if 'programming_skills' in filtered_words:
        filtered_words = [word for word in filtered_words if word not in programming_skills]
    if 'time_management' in filtered_words:
        filtered_words = [word for word in filtered_words if word not in time_management]
    if 'taking_initiative_skills' in filtered_words:
        filtered_words = [word for word in filtered_words if word not in taking_initiative]

    all_words.extend(filtered_words)

# Count the most common words and phrases
most_common_words = Counter(all_words).most_common(17)

# Display clusters of common skills or themes
print("\nIdentified Common Skills or Themes:")
for skill, count in most_common_words:
    print(f"{skill.replace('_', ' ').capitalize()} (mentioned {count} times)")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Index(['Responder ID', 'Terms', 'Coll', 'Major', 'Conc', 'Class BOT',
       'Citizenship', 'Coop #',
       'What skills do you need to develop or improve to better prepare you for your next position?'],
      dtype='object')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]


Identified Common Skills or Themes:
Communication skills (mentioned 254 times)
Excel (mentioned 197 times)
Programming skills (mentioned 169 times)
Time management (mentioned 147 times)
Ask questions skills (mentioned 125 times)
Finance skills (mentioned 114 times)
Presentation skills (mentioned 109 times)
Technical skills (mentioned 81 times)
Task completion (mentioned 58 times)
Organizational skills (mentioned 52 times)
Give new ideas (mentioned 49 times)
Public speaking (mentioned 48 times)
Data analysis skills (mentioned 45 times)
Make (mentioned 39 times)
Ability (mentioned 38 times)
Use (mentioned 37 times)
Believe (mentioned 36 times)


** Generate csv with Findings - for KPI 1

In [ ]:
import pandas as pd
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords
from transformers import BertTokenizer, BertModel
import torch

# Download the stopwords from NLTK
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Load the cleaned data from the provided Excel file
file_student = "/content/drive/MyDrive/M.S. BUSINESS ANALYTICS/MIS612/CO-OP SURVEY GROUP PROJECT/Cleaned_Data/Cleaned_Student_2020_Responses.xlsx"
sheets_to_read = ["College Specific Questions"]

# Read only the necessary sheet from the Excel file using the first row as headers
sheets_student = pd.read_excel(file_student, sheet_name=sheets_to_read, header=0)
college_specific_questions_df = sheets_student["College Specific Questions"]

# Columns to be included in the new Excel sheet
metadata_columns = ["Responder ID", "Terms", "Coll", "Major", "Conc", "Class BOT", "Citizenship", "Coop #"]

# Verify the columns exist
for col in metadata_columns:
    if col not in college_specific_questions_df.columns:
        raise ValueError(f"Column '{col}' not found in the DataFrame. Please check the column names.")

# Extract the data needed
metadata_df = college_specific_questions_df[metadata_columns]
column_name = "What skills do you need to develop or improve to better prepare you for your next position?"
improve_skills_responses = college_specific_questions_df[column_name].dropna().astype(str).tolist()

# Define the list of top 18 skills identified
skills_list = [
    "communication_skills",
    "excel",
    "programming_skills",
    "finance_skills",
    "presentation_skills",
    "ask_questions_skills",
    "time_management",
    "technical_skills",
    "task_completion",
    "organizational_skills",
    "give_new_ideas",
    "confidence",
    "data_analysis_skills",
    "problem_solving_skills",
    "marketing",
    "attention_to_detail",
    "other",
    "taking_initiative"
]

# Define synonyms for common phrases to be treated as equivalent
skills_synonyms = {
    "data_analysis_skills": ["data analysis", "data analytics", "analyzing data", "data examination", "analytical skills", "data", "analytical"],
    "finance_skills": ["financial analysis", "financial", "finance", "budgeting", "financial planning", "accounting"],
    "problem_solving_skills": ["negotiation", "logical reasoning", "reasoning", "creative thinking", "problem solving", "critical thinking", "troubleshooting", "attention to detail", "analytical thinking"],
    "ask_questions_skills": ["help", "asking", "ask", "ask questions", "ask more questions", "ask question", "asking questions", "asking a qu"],
    "communication_skills": ["talking", "talk", "communication", "verbal communication", "written communication", "verbal", "written", "communication skills", "express", "writing skills", "writing", "communications"],
    "presentation_skills": ["speaking", "speak", "presenter", "story telling", "storytelling", "public speaking", "presenting", "present", "presentation"],
    "give_new_ideas": ["thoughts", "convey my thoughts", "convey", "thoughts", "new ideas", "new idea", "ideas", "give ideas", "idea", "suggestion", "new suggestions", "suggestions"],
    "confidence": ["confidence", "confident", "confidently", "confidence skills"],
    "attention_to_detail": ["listening", "active listening", "actively listening", "attention to detail", "attention", "details", "detail", "attention to", "detail oriented", "detailoriented"],
    "task_completion": ["finish", "finish projects", "complete task", "complete tasks", "task completion", "task", "tasks", "following instruction", "following instructions", "instructions"],
    "organizational_skills": ["organizational skills", "organized", "organization", "organize", "project management"],
    "marketing": ["market research", "creative", "marketing courses", "marketing", "marketing skills", "social media"],
    "technical_skills": ["research skills", "research", "technical skills", "technical"],
    "programming_skills": ["technological", "computer skills", "computer", "tableau", "sap", "scripting", "microsoft", "web design", "coding", "software", "softwares", "python", "sql", "power bi", "programming"],
    "time_management": ["time efficiency", "time management", "better time"],
    "taking_initiative": ["taking initiative", "taking initiatives", "initiative", "proactive", "initiatives"]
}

# Create a new DataFrame with the metadata columns and the additional skill columns, initializing with zeros
extended_df = metadata_df.copy()
for skill in skills_list:
    extended_df[skill] = 0

# Iterate over each response and update the corresponding skill columns in the DataFrame
for idx, response in enumerate(improve_skills_responses):
    response = response.lower()

    # Check if each skill is mentioned and update the corresponding column
    skill_found = False
    for skill, synonyms in skills_synonyms.items():
        if any(synonym in response for synonym in synonyms):
            extended_df.at[idx, skill] = 1
            skill_found = True

    # Check for Excel separately as it does not have a synonyms list
    if "excel" in response:
        extended_df.at[idx, "excel"] = 1
        skill_found = True

    # If no skill was found, mark the "other" column as 1
    if not skill_found:
        extended_df.at[idx, "other"] = 1

# Add a sum row at the end of each of the newly added skill columns
sum_row = extended_df[skills_list].sum()
sum_row = pd.Series(sum_row, index=extended_df.columns)
sum_row[metadata_columns] = "Total"
extended_df = pd.concat([extended_df, pd.DataFrame([sum_row])], ignore_index=True)

# Save the updated DataFrame to a new Excel file
output_excel_path = "/content/drive/MyDrive/M.S. BUSINESS ANALYTICS/MIS612/CO-OP SURVEY GROUP PROJECT/Objective 1/Student_Skills_Analysis.xlsx"
extended_df.to_excel(output_excel_path, index=False)
print(f"Extended data with skills analysis saved to {output_excel_path}")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-10-e0b0e0168d1e>:104: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Total' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  sum_row[metadata_columns] = "Total"


Extended data with skills analysis saved to /content/drive/MyDrive/M.S. BUSINESS ANALYTICS/MIS612/CO-OP SURVEY GROUP PROJECT/Objective 1/Student_Skills_Analysis.xlsx


## **Text Mining** File: Cleaned_Employer_2020_Responses  
Objective: Skill Gaps Between International and Domestic Students

KPI 2: Weaknesses, by employers




**Step 4 - Extract Common Skills by students - for KPI 1

In [ ]:
import pandas as pd
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords
from transformers import BertTokenizer, BertModel
import torch

# Download the stopwords from NLTK
nltk.download('stopwords')
employer_stop_words = set(stopwords.words('english'))

# Load the cleaned data from the provided Excel file
file_employer = "/content/drive/MyDrive/M.S. BUSINESS ANALYTICS/MIS612/CO-OP SURVEY GROUP PROJECT/Cleaned_Data/Cleaned_Employer_2020_Responses.xlsx"
sheets_to_read = ["Weaknesses"]

# Read only the necessary sheet from the Excel file using the first row as headers
sheets_employer = pd.read_excel(file_employer, sheet_name=sheets_to_read, header=0)
weaknesses_df = sheets_employer["Weaknesses"]

# Print the column names to verify
print(weaknesses_df.columns)

# Extract the correct column: "What are the student's weaknesses/areas for improvement?"
# Please make sure the column name matches exactly with what is in your Excel sheet
employer_column_name = "What are the student's weaknesses/areas for improvement?"
employer_weaknesses_responses = weaknesses_df["What are the student's weaknesses/areas for improvement?"].dropna().astype(str).tolist()

# Generate embeddings using BERT for contextual understanding
# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Tokenize and encode the responses using BERT tokenizer
emp_encoded_inputs = tokenizer(
    employer_weaknesses_responses,
    padding=True,
    truncation=True,
    return_tensors='pt'
)

# Generate embeddings using BERT
with torch.no_grad():
    emp_outputs = model(**emp_encoded_inputs)
    # Use the CLS token representation as the embedding for each response
    employer_embeddings = emp_outputs.last_hidden_state[:, 0, :]

# Convert embeddings from GPU tensor to CPU if needed
employer_embeddings_cpu = employer_embeddings.cpu().numpy() if employer_embeddings.is_cuda else employer_embeddings.numpy()

# Analyze responses to identify common weaknesses, excluding stopwords and specific irrelevant terms
employer_excluded_words = {'still','understanding','often', 'little','great','one','however','new','give','come','see','take','takes','taking','times','time','sometime','sometimes','question','project','projects','x000d','team','skills', 'skill','student','area','areas','weakness','weaknesses','improve','improvement', 'work', 'improve', 'need','needs','continue','experience','learn', 'enhance', 'develop', 'would', 'coop', 'response', 'error', 'better', 'think', 'position', 'next', 'should', 'like', 'want', 'will', 'feel', 'could', 'also', 'get', 'management','co','op','knowledge','able','time','well','help','working','always','thing','prepare','things','something','somethings','different'}
employer_combined_phrases = ["taking initiative","taking initiatives","project management", "time management", "attention to detail", "public speaking","complete task","complete tasks","new ideas","new idea"]

# Define synonyms for common phrases to be treated as equivalent
emp_data_analysis_skills = ["data analysis", "data analytics", "analyzing data", "data examination","analytical skills","data","analytical"]
emp_finance_skills = ["financial analysis","financial", "finance", "budgeting", "financial planning", "accounting"]
emp_problem_solving_skills = ["negotiation","logical reasoning","reasoninig","creative thinking","problem solving", "critical thinking", "troubleshooting", "attention to detail", "analytical thinking"]
emp_ask_questions_skills = ["help","asking","ask","ask questions","ask more questions","ask question","asking questions","asking a","questions","question"]
emp_communication_skills = ["talking","talk","communication","verbal communication","written communication","verbal","written","communication skills","express","writing skills","writing","communications"]
emp_presentation_skills = ["speaking","speak","presenter","story telling","storytelling","public speaking","presenting","present","presentation"]
emp_give_new_ideas = ["thoughts","convey my thoughts","convey","thoughts","new ideas","new idea","ideas","give ideas","idea","suggestion","new suggestions","suggestions"]
emp_confidence = ["confidence","confident","confidently","confidence skills"]
emp_attention_to_detail = ["listening","active listening", "actively listening","attention to detail","attention","details","detail","attention to","detail oriented","detailoriented"]
emp_task_completion = ["finish","finish projects","complete task","complete tasks","task completion","task","tasks","following instruction","following instructions","instructions"]
emp_organizational_skills = ["organizational skills","organized","organization","organize","project management"]
emp_marketing = ["market research","creative","marketing courses","marketing","marketing skills","social media",]
emp_technical_skills = ["research skills","research","technical skills","technical",]
emp_programming_skills = ["technological","computer skills","computer", "tableau", "sap","scripting","microsoft","web design","coding","software","softwares","python","sql","power bi","programming"]
emp_time_management = ["time efficiency","time management","better time","time"]
emp_taking_initiative = ["taking initiative","taking initiatives","initiative","proactive","initiatives"]

# Update combined phrases list to include more groupings
employer_combined_phrases.extend(emp_data_analysis_skills)
employer_combined_phrases.extend(emp_finance_skills)
employer_combined_phrases.extend(emp_problem_solving_skills)
employer_combined_phrases.extend(emp_ask_questions_skills)
employer_combined_phrases.extend(emp_communication_skills)
employer_combined_phrases.extend(emp_presentation_skills)
employer_combined_phrases.extend(emp_give_new_ideas)
employer_combined_phrases.extend(emp_confidence)
employer_combined_phrases.extend(emp_attention_to_detail)
employer_combined_phrases.extend(emp_task_completion)
employer_combined_phrases.extend(emp_organizational_skills)
employer_combined_phrases.extend(emp_marketing)
employer_combined_phrases.extend(emp_technical_skills)
employer_combined_phrases.extend(emp_programming_skills)
employer_combined_phrases.extend(emp_time_management)
employer_combined_phrases.extend(emp_taking_initiative)


# Tokenize all responses to find common words and phrases
employer_all_words = []
for response in employer_weaknesses_responses:
    # Replace combined phrases first
    response = response.lower()
    for phrase in employer_combined_phrases:
        response = re.sub(rf'\b{re.escape(phrase)}\b', phrase.replace(' ', '_'), response)  # Replace combined phrases with underscores
    # Replace synonyms for data analysis
    for synonym in emp_data_analysis_skills:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'emp_data_analysis_skills', response)
    for synonym in emp_finance_skills:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'emp_finance_skills', response)
    for synonym in emp_problem_solving_skills:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'emp_problem_solving_skills', response)
    for synonym in emp_ask_questions_skills:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'emp_ask_questions_skills', response)
    for synonym in emp_communication_skills:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'emp_communication_skills', response)
    for synonym in emp_presentation_skills:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'emp_presentation_skills', response)
    for synonym in emp_give_new_ideas:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'emp_give_new_ideas', response)
    for synonym in emp_confidence:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'emp_confidence', response)
    for synonym in emp_attention_to_detail:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'emp_attention_to_detail', response)
    for synonym in emp_task_completion:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'emp_task_completion', response)
    for synonym in emp_organizational_skills:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'emp_organizational_skills', response)
    for synonym in emp_marketing:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'emp_marketing', response)
    for synonym in emp_technical_skills:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'emp_technical_skills', response)
    for synonym in emp_programming_skills:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'emp_programming_skills', response)
    for synonym in emp_time_management:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'emp_time_management', response)
    for synonym in emp_taking_initiative:
        response = re.sub(rf'\b{re.escape(synonym)}\b', 'emp_taking_initiative', response)
    # Tokenize the response
    employer_words = re.findall(r'\b\w+_?\w*\b', response)  # Extract all words and combined phrases
    employer_filtered_words = [word for word in employer_words if word not in employer_stop_words and word not in employer_excluded_words]
    # Remove subset terms if the grouped term is already present
    if 'emp_data_analysis_skills' in employer_filtered_words:
        employer_filtered_words = [word for word in employer_filtered_words if word not in emp_data_analysis_skills]
    if 'emp_finance_skills' in employer_filtered_words:
        employer_filtered_words = [word for word in employer_filtered_words if word not in emp_finance_skills]
    if 'emp_problem_solving_skills' in employer_filtered_words:
        employer_filtered_words = [word for word in employer_filtered_words if word not in emp_problem_solving_skills]
    if 'emp_ask_questions_skills' in employer_filtered_words:
        employer_filtered_words = [word for word in employer_filtered_words if word not in emp_ask_questions_skills]
    if  'emp_communication_skills' in employer_filtered_words:
        employer_filtered_words = [word for word in employer_filtered_words if word not in emp_communication_skills]
    if 'emp_presentation_skills' in employer_filtered_words:
        employer_filtered_words = [word for word in employer_filtered_words if word not in emp_presentation_skills]
    if 'emp_give_new_ideas' in employer_filtered_words:
        employer_filtered_words = [word for word in employer_filtered_words if word not in emp_give_new_ideas]
    if 'emp_confidence' in employer_filtered_words:
        employer_filtered_words = [word for word in employer_filtered_words if word not in emp_confidence]
    if  'emp_attention_to_detail' in employer_filtered_words:
        employer_filtered_words = [word for word in employer_filtered_words if word not in emp_attention_to_detail]
    if 'emp_task_completion' in employer_filtered_words:
        employer_filtered_words = [word for word in employer_filtered_words if word not in emp_task_completion]
    if 'emp_organizational_skills' in employer_filtered_words:
        employer_filtered_words = [word for word in employer_filtered_words if word not in emp_organizational_skills]
    if 'emp_marketing' in employer_filtered_words:
        employer_filtered_words = [word for word in employer_filtered_words if word not in emp_marketing]
    if 'emp_technical_skills' in employer_filtered_words:
        employer_filtered_words = [word for word in employer_filtered_words if word not in emp_technical_skills]
    if 'emp_programming_skills' in employer_filtered_words:
        employer_filtered_words = [word for word in employer_filtered_words if word not in emp_programming_skills]
    if 'emp_time_management' in employer_filtered_words:
        employer_filtered_words = [word for word in employer_filtered_words if word not in emp_time_management]
    if 'emp_taking_initiative' in employer_filtered_words:
        employer_filtered_words = [word for word in employer_filtered_words if word not in emp_taking_initiative]

    employer_all_words.extend(employer_filtered_words)
# Count the most common words and phrases
employer_most_common_words = Counter(employer_all_words).most_common(10)

# Display clusters of common skills or themes
print("\nIdentified Common Weaknesses or Themes:")
for weakness, count in employer_most_common_words:
    print(f"{weakness.replace('_', ' ').capitalize()} (mentioned {count} times)")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Index(['Responder ID', 'Terms', 'Coll', 'Major', 'Conc 1', 'Class BOT',
       'Citizenship', 'Coop #',
       'What are the student's weaknesses/areas for improvement?'],
      dtype='object')


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



Identified Common Weaknesses or Themes:
Emp task completion (mentioned 157 times)
Emp time management (mentioned 133 times)
Emp communication skills (mentioned 118 times)
Emp ask questions skills (mentioned 107 times)
Emp taking initiative (mentioned 94 times)
Emp give new ideas (mentioned 71 times)
Emp presentation skills (mentioned 68 times)
Emp confidence (mentioned 61 times)
Issues (mentioned 34 times)
Business (mentioned 33 times)


** Generate csv with Findings - for KPI 1

In [ ]:
import pandas as pd
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords
from transformers import BertTokenizer, BertModel
import torch

# Download the stopwords from NLTK
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Load the cleaned data from the provided Excel file
file_employer = "/content/drive/MyDrive/M.S. BUSINESS ANALYTICS/MIS612/CO-OP SURVEY GROUP PROJECT/Cleaned_Data/Cleaned_Employer_2020_Responses.xlsx"
sheets_to_read = ["Weaknesses"]

# Read only the necessary sheet from the Excel file using the first row as headers
sheets_employer = pd.read_excel(file_employer, sheet_name=sheets_to_read, header=0)
weaknesses_df = sheets_employer["Weaknesses"]

# Columns to be included in the new Excel sheet
emp_metadata_columns = ["Responder ID", "Terms", "Coll", "Major", "Conc 1", "Class BOT", "Citizenship", "Coop #"]

# Verify the columns exist
for col in emp_metadata_columns:
    if col not in weaknesses_df.columns:
        raise ValueError(f"Column '{col}' not found in the DataFrame. Please check the column names.")

# Extract the data needed
emp_metadata_df = weaknesses_df[emp_metadata_columns]
emp_column_name = "What are the student's weaknesses/areas for improvement?"
improve_weaknesses_responses = weaknesses_df[emp_column_name].dropna().astype(str).tolist()

# Define the list of top 18 skills identified
weaknesses_list = [
    "emp_communication_skills",
    "emp_excel",
    "emp_programming_skills",
    "emp_finance_skills",
    "emp_presentation_skills",
    "emp_ask_questions_skills",
    "emp_time_management",
    "emp_technical_skills",
    "emp_task_completion",
    "emp_organizational_skills",
    "emp_give_new_ideas",
    "emp_confidence",
    "emp_data_analysis_skills",
    "emp_problem_solving_skills",
    "emp_marketing",
    "emp_attention_to_detail",
    "emp_other",
    "emp_taking_initiative"
]

# Define synonyms for common phrases to be treated as equivalent
weaknesses_synonyms = {
    "emp_data_analysis_skills": ["data analysis", "data analytics", "analyzing data", "data examination", "analytical skills", "data", "analytical"],
    "emp_finance_skills": ["financial analysis", "financial", "finance", "budgeting", "financial planning", "accounting"],
    "emp_problem_solving_skills": ["negotiation", "logical reasoning", "reasoning", "creative thinking", "problem solving", "critical thinking", "troubleshooting", "attention to detail", "analytical thinking"],
    "emp_ask_questions_skills": ["help", "asking", "ask", "ask questions", "ask more questions", "ask question", "asking questions", "asking a qu"],
    "emp_communication_skills": ["talking", "talk", "communication", "verbal communication", "written communication", "verbal", "written", "communication skills", "express", "writing skills", "writing", "communications"],
    "emp_presentation_skills": ["speaking", "speak", "presenter", "story telling", "storytelling", "public speaking", "presenting", "present", "presentation"],
    "emp_give_new_ideas": ["thoughts", "convey my thoughts", "convey", "thoughts", "new ideas", "new idea", "ideas", "give ideas", "idea", "suggestion", "new suggestions", "suggestions"],
    "emp_confidence": ["confidence", "confident", "confidently", "confidence skills"],
    "emp_attention_to_detail": ["listening", "active listening", "actively listening", "attention to detail", "attention", "details", "detail", "attention to", "detail oriented", "detailoriented"],
    "emp_task_completion": ["finish", "finish projects", "complete task", "complete tasks", "task completion", "task", "tasks", "following instruction", "following instructions", "instructions"],
    "emp_organizational_skills": ["organizational skills", "organized", "organization", "organize", "project management"],
    "emp_marketing": ["market research", "creative", "marketing courses", "marketing", "marketing skills", "social media"],
    "emp_technical_skills": ["research skills", "research", "technical skills", "technical"],
    "emp_programming_skills": ["technological", "computer skills", "computer", "tableau", "sap", "scripting", "microsoft", "web design", "coding", "software", "softwares", "python", "sql", "power bi", "programming"],
    "emp_time_management": ["time efficiency", "time management", "better time"],
    "emp_taking_initiative": ["taking initiative", "taking initiatives", "initiative", "proactive", "initiatives"]
}

# Create a new DataFrame with the metadata columns and the additional skill columns, initializing with zeros
emp_extended_df = emp_metadata_df.copy()
for weakness in weaknesses_list:
    emp_extended_df[weakness] = 0

# Iterate over each response and update the corresponding skill columns in the DataFrame
for idx, emp_response in enumerate(improve_weaknesses_responses):
    emp_response = emp_response.lower()

    # Check if each skill is mentioned and update the corresponding column
    weakness_found = False
    for weakness, synonyms in weaknesses_synonyms.items():
        if any(synonym in emp_response for synonym in synonyms):
            emp_extended_df.at[idx, weakness] = 1
            weakness_found = True

    # Check for Excel separately as it does not have a synonyms list
    if "excel" in emp_response:
        emp_extended_df.at[idx, "emp_excel"] = 1
        weakness_found = True

    # If no skill was found, mark the "emp_other" column as 1
    if not weakness_found:
        emp_extended_df.at[idx, "emp_other"] = 1

# Add a sum row at the end of each of the newly added skill columns
emp_sum_row = emp_extended_df[weaknesses_list].sum()
emp_sum_row = pd.Series(emp_sum_row, index=emp_extended_df.columns)
emp_sum_row[emp_metadata_columns] = "Total"
emp_extended_df = pd.concat([emp_extended_df, pd.DataFrame([emp_sum_row])], ignore_index=True)

# Save the updated DataFrame to a new Excel file
emp_output_excel_path = "/content/drive/MyDrive/M.S. BUSINESS ANALYTICS/MIS612/CO-OP SURVEY GROUP PROJECT/Objective 1/Employer_Skills_Analysis.xlsx"
emp_extended_df.to_excel(emp_output_excel_path, index=False)
print(f"Extended data with employer skills analysis saved to {emp_output_excel_path}")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-22-5354b5edc282>:104: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Total' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  emp_sum_row[emp_metadata_columns] = "Total"


Extended data with employer skills analysis saved to /content/drive/MyDrive/M.S. BUSINESS ANALYTICS/MIS612/CO-OP SURVEY GROUP PROJECT/Objective 1/Employer_Skills_Analysis.xlsx
